In [1]:
import functools
import time, timeit

from IPython.display import HTML, Image
import gym

import brax.v1 as brax

from brax.v1.physics import config_pb2


from brax.v1 import envs
from brax.v1.io import html
import jax
from jax import numpy as jnp
from jax import random

from builder import *
from builder import distribute
from builder.spawn import Spawner

import time
import json

In [2]:
seed = 42
key = random.PRNGKey(42)

2023-06-10 16:13:32.400682: W external/xla/xla/service/platform_util.cc:198] unable to create StreamExecutor for CUDA:0: failed initializing StreamExecutor for CUDA device ordinal 0: INTERNAL: failed call to cuDevicePrimaryCtxRetain: CUDA_ERROR_DEVICE_UNAVAILABLE: CUDA-capable device(s) is/are busy or unavailable
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [3]:
separation = 5
iterations = 25
env_dims = [(5, 5), (10, 10), (15, 15), (20, 20), (25, 25), (30, 30), (40, 40), (60, 60), (100, 100)]

In [4]:
poisson_times = dict()
random_times = dict()

In [5]:
def add_agent(config, agent_mass=1.0, radius=0.5):
    # Add the agent body
    agent = config.bodies.add(name='agent')
    sph = agent.colliders.add().sphere
    sph.radius = radius
    agent.mass = agent_mass
    # agent.damping = 1e-2
    # agent.friction = 0.6

In [6]:
def create_config():
    # Define the environment config.
    sphere_maze = brax.Config(dt=0.01, substeps=2, dynamics_mode='pbd')
    


    sphere_maze.gravity.x = 0.0
    sphere_maze.gravity.y = 0.0
    sphere_maze.gravity.z = -9.8

    # Add the ground, a frozen (immovable) infinite plane
    ground = sphere_maze.bodies.add(name='ground')
    ground.frozen.all = True
    plane = ground.colliders.add().plane
    plane.SetInParent()  # for setting an empty oneof

    # Add the agent body
    add_agent(sphere_maze)

    # # Add the cubes.
    # add_objects(sphere_maze, key, object_count=object_count)
    # distribute_objects(sphere_maze, key, object_count=object_count)

    return sphere_maze

In [7]:
for j in range(iterations):
    print("Iteration: ", j)
    for dim in env_dims:
        print("Env dim: ", dim)
        config = create_config()

        st = time.time()
        # Distribute the objects
        spawner = Spawner(env_dim=dim)
        spawner.spawn_objects(config, key, dim, separation, obj_type="box", method="poisson")
        poisson_time = time.time() - st

        # Distribute the objects
        spawner = Spawner(env_dim=dim)
        spawner.spawn_objects(config, key, dim, separation, obj_type="box", method="separated")
        random_time = time.time() - st - poisson_time


        # Add to dicts
        if dim[0] not in poisson_times:
            poisson_times[dim[0]] = [poisson_time]
        else:
            poisson_times[dim[0]].append(poisson_time)

        if dim[0] not in random_times:
            random_times[dim[0]] = [random_time]
        else:
            random_times[dim[0]].append(random_time)

Iteration:  0
Env dim:  (5, 5)
Env dim:  (10, 10)
Env dim:  (15, 15)
Env dim:  (20, 20)
Env dim:  (25, 25)
Env dim:  (30, 30)
Env dim:  (40, 40)
Env dim:  (60, 60)
Env dim:  (100, 100)
Iteration:  1
Env dim:  (5, 5)
Env dim:  (10, 10)
Env dim:  (15, 15)
Env dim:  (20, 20)
Env dim:  (25, 25)
Env dim:  (30, 30)
Env dim:  (40, 40)
Env dim:  (60, 60)
Env dim:  (100, 100)
Iteration:  2
Env dim:  (5, 5)
Env dim:  (10, 10)
Env dim:  (15, 15)
Env dim:  (20, 20)
Env dim:  (25, 25)
Env dim:  (30, 30)
Env dim:  (40, 40)
Env dim:  (60, 60)
Env dim:  (100, 100)
Iteration:  3
Env dim:  (5, 5)
Env dim:  (10, 10)
Env dim:  (15, 15)
Env dim:  (20, 20)
Env dim:  (25, 25)
Env dim:  (30, 30)
Env dim:  (40, 40)
Env dim:  (60, 60)
Env dim:  (100, 100)
Iteration:  4
Env dim:  (5, 5)
Env dim:  (10, 10)
Env dim:  (15, 15)
Env dim:  (20, 20)
Env dim:  (25, 25)
Env dim:  (30, 30)
Env dim:  (40, 40)
Env dim:  (60, 60)
Env dim:  (100, 100)
Iteration:  5
Env dim:  (5, 5)
Env dim:  (10, 10)
Env dim:  (15, 15)
Env di

In [8]:
# Save dicitonaries to json files
with open('poisson_times.json', 'w') as fp:
    json.dump(poisson_times, fp)

with open('random_times.json', 'w') as fp:
    json.dump(random_times, fp)

In [9]:
# HTML(html.render(sys, qps, height=800))

In [10]:
# html.save_html('v1.html', sys, qps)